# Формальные степенные ряды

Каждый ряд $f(x)$ -- это бесконечная последовательность коэффициентов. То есть ряд
$$
f(x) = f_0 + f_1 x + f_2 x^2 + f_3 x^3 + ...
$$
есть последовательность
$$
[f_0, f_1, f_2, f_3, ...].
$$

Последовательность мы представляем как структуру, часть вычислений которой отложенные. Так описанная выше последовательность есть `Cons(f_0, Cons(f_1, Cons(f_2, Cons(f_3, Thunk))))`. Первые четыре элемента вычислены, а для вычисления пятого надо вызвать функцию, которая этот пятый элемент посчитает.

Cons-ячейка -- это пара, которая даёт информацию о том, каково значение текущего коэффициента, и как вычислить значения всех остальных коэффициентов.

In [56]:
import Base: show, +, *, /, -
using Latexify

In [2]:
"Объединяем разные структуры под одним типом"
abstract type Series{T <: Number}
end


"Cons-ячейка -- это значение текущего коэффициента и ссылка на оставшийся хвост"
mutable struct Cons{T <: Number} <: Series{T}
    head::T
    tail::Series{T}
    
    function Cons{T}(head::T, tail::Cons{T}) where T <: Number
        return new(head, tail)
    end
    
    function Cons{T}() where T <: Number
        return new()
    end
end


"Memo -- обёртка, которая при вычислении меняет родительскую ссылку"
struct Memo{T <: Number} <: Series{T}
    parent::Cons{T}
    child::Series{T}
    
    function Memo{T}(parent::Cons{T}, child::Series{T}) where T <: Number
        if isa(child, Memo)
            return Memo{T}(parent, child.child)
        else
            return new(parent, child)
        end
    end
end


"Вычисление следующего элемента."
function force end

function force(s::Cons{T})::Cons{T} where T <: Number
    return s
end

function force(s::Memo{T})::Cons{T} where T <: Number
    return s.parent.tail = force(s.child)
end

force (generic function with 2 methods)

In [3]:
function consmemo end

function consmemo(head::T, tail::Cons{T})::Series{T} where T <: Number
    return Cons{T}(head, tail)
end

function consmemo(head::T, tail::Series{T})::Series{T} where T <: Number
    self = Cons{T}()
    self.head = head
    self.tail = Memo{T}(self, tail)
    return self
end

"Мемоизация степенного ряда"
function memoize(s::Series{T})::Cons{T} where T <: Number
    c = force(s) :: Cons{T}
    return consmemo(c.head, c.tail)
end

memoize

In [4]:
struct SeriesGenerator{T <: Number} <: Series{T}
    index::Int
    generator::Function
end


"Конструктор степенного ряда из генерирующей функции"
function generate(::Type{T}, gen::Function)::Series{T} where T <: Number
    return SeriesGenerator{T}(0, gen)
end


function force(s::SeriesGenerator{T})::Cons{T} where T <: Number
    gen = s.generator
    return consmemo(T(gen(s.index)), SeriesGenerator{T}(s.index + 1, gen))
end


struct SeriesVector{T <: Number} <: Series{T}
    index::Int
    data::Vector{T}
    cycled::Bool
end

"Конструктор степенного ряда из вектора"
function series(data::Vector{T}; cycled::Bool=false)::Series{T} where T <: Number
    return SeriesVector{T}(0, data, cycled)
end

function force(s::SeriesVector{T})::Cons{T} where T <: Number
    n = length(s.data)
    i = s.index
    if s.cycled
        return consmemo(s.data[i % n + 1], SeriesVector{T}(i + 1, s.data, s.cycled))
    else
        return consmemo((i >= n) ? T(0) : s.data[i + 1], SeriesVector{T}(i + 1, s.data, s.cycled))
    end
end

force (generic function with 4 methods)

In [5]:
"Взять первые n коэффициентов и вернуть их в векторе"
function take(s::Series{T}, n::Int)::Vector{T} where T <: Number
    vec = Vector{T}(undef, n)
    
    for i in 1:n
        c = force(s)::Cons{T}
        vec[i] = c.head
        s = c.tail
    end
    
    return vec
end

take

In [6]:
"Преобразовать часть степенного ряда в выражение"
function series_expr(s::Series{T}, n::Int=9, var::Symbol=:x)::Expr where T <: Number
    cons = force(s)::Cons{T}
    args = Any[cons.head]
    s = cons.tail

    for i in 1:n
        cons = force(s)::Cons{T}
        coef = cons.head
        s = cons.tail

        term = (i == 1) ? var : :($var ^ $i)
        term = (coef == 1) ? term : :($coef * $term)
        push!(args, term)
    end
    return Expr(:call, :+, args..., :...)
end

"Красивое представление степенного ряда в LaTeX"
function show(io::IO, ::MIME"text/latex", s::Series{T})::Nothing where T <: Number
    println(io, latexify(series_expr(s)))
end

function show(io::IO, ::MIME"text/plain", s::Series{T})::Nothing where T <: Number
    println(io, latexify(series_expr(s)))
end

show (generic function with 263 methods)

Далее ряды будем представлять следующим образом:
$$
f(x) = f_0 + f_1 x + f_2 x^2 + \ldots = f_h + x \cdot f_t(x),
$$

где $f_h = f_0$ есть "голова"(head), а $f_t(x) = f_1 + f_2 x + f_3 x^2 + \ldots$ есть "хвост"(tail)

### Сложение степенных рядов
Пусть есть ряды $f(x) = f_h + x \cdot f_t(x)$ и $g(x) = g_h + x \cdot g_t(x)$, тогда:

$$
f(x) + g(x) = \left(f_h + g_h\right) + x \cdot \left(f_t(x) + g_t(x)\right)
$$

In [7]:
struct SeriesAdd{T <: Number} <: Series{T}
    first::Series{T}
    second::Series{T}
end


function +(s1::Series{T}, s2::Series{T})::Series{T} where T <: Number
    return SeriesAdd{T}(s1, s2)
end


function force(s::SeriesAdd{T})::Cons{T} where T <: Number
    first = force(s.first)
    second = force(s.second)
    return consmemo(first.head + second.head, SeriesAdd{T}(first.tail, second.tail))
end

force (generic function with 5 methods)

### Умножение ряда на число
Пусть есть число $a$ и ряд $f(x) = f_h + x \cdot f_t(x)$, тогда:

$$
a \cdot f(x) = \left(a \cdot f_h \right) + x \cdot \left(a \cdot f_t(x)\right)
$$

In [57]:
struct SeriesScale{T <: Number} <: Series{T}
    scaler::T
    series::Series{T}
end


function *(a::T, f::Series{T})::Series{T} where T <: Number
    return SeriesScale{T}(a, f)
end


function -(f::Series{T})::Series{T} where T <: Number
    return SeriesScale{T}(T(-1), f)
end


function force(s::SeriesScale{T})::Cons{T} where T <: Number
    c = force(s.series)::Cons{T}
    return consmemo(s.scaler * c.head, SeriesScale{T}(s.scaler, c.tail))
end

force (generic function with 13 methods)

### Умножение рядов

Пусть есть ряды $f(x) = f_h + x f_t(x)$ и $g(x)$, тогда:

$$
f(x) \cdot g(x) = \left(f_h \cdot g(x)\right) + x \cdot \left(f_t(x) \cdot g(x)\right)
$$

In [9]:
struct SeriesMul{T <: Number} <: Series{T}
    fst::Series{T}
    snd::Series{T}
end


function *(f::Series{T}, g::Series{T})::Series{T} where T <: Number
    return SeriesMul{T}(f, g)
end


function force(s::SeriesMul{T})::Cons{T} where T <: Number
    fst = force(s.fst)
    snd = force(s.snd)
    return force(fst.head * snd + consmemo(T(0), snd * fst.tail))
end

force (generic function with 7 methods)

### Подстановка
Пусть есть ряды $f(x) = f_h + x f_t(x)$ и $g(x) = x g_t(x)$, тогда:

$$
f(g(x)) = f_h + g(x) \cdot f_t(g(x)) = f_h + x \cdot g_t(x) \cdot f_t(g(x))
$$

Наличие $x$ во втором слагаемом даёт возможность отложить вычисление этого второго слагаемого.

In [10]:
struct SeriesApply{T <: Number} <: Series{T}
    fun::Series{T}
    arg::Series{T}
end


function (f::Series{T})(g::Series{T})::Series{T} where T <: Number
    return SeriesApply{T}(f, g)
end


function force(s::SeriesApply{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    arg.head == 0 || throw(ErrorException("non-zero head in application"))

    fun = force(s.fun)::Cons{T}
    return consmemo(fun.head, arg.tail * fun.tail(arg))
end

force (generic function with 8 methods)

### Обращение ряда (в смысле деления)

Пусть есть ряд $f(x) = f_h + x \cdot f_t(x)$, тогда:

$$
\frac{1}{f(x)} = \frac{1}{f_h + x \cdot f_t(x)} = \frac{1}{f_h} \cdot \frac{1}{1 + x \frac{f_t(x)}{f_h}}
= \frac{1}{f_h} \cdot r\left(-x \frac{f_t(x)}{f_h}\right),
$$
где $r(x) = 1 + x + x^2 + x^3 + \ldots$

In [11]:
"Обращение ряда (в смысле деления)"
function recip(s::Series{T})::Series{T} where T <: Number
    c = force(s)::Cons{T}
    c.head == 0 && throw(ErrorException("zero head in reciprocation"))

    ones = series(T[1])
    rh = 1 / c.head
    return rh * ones(consmemo(T(0), -rh * c.tail))
end

recip

### Деление рядов

$$
\frac{f(x)}{g(x)} = f(x) \cdot \frac{1}{g(x)}
$$

In [12]:
function /(s1::Series{T}, s2::Series{T})::Series{T} where T <: Number
    den = force(s2)::Cons{T}
    if den.head == 0
        num = force(s1)::Cons{T}
        num.head == 0 || throw(ErrorException("zero head in reciprocation"))
        return num.tail / den.tail
    else
        return s1 * recip(den)
    end
end

/ (generic function with 107 methods)

In [32]:
"Решить уравнение f(g(x)) = h(x)"
struct SeriesSolve{T <: Number} <: Series{T}
    arg::Series{T}
    res::Series{T}
end


function solve(g::Series{T}, h::Series{T}=series(T[0, 1])) where T <: Number
    return SeriesSolve{T}(g, h)
end


function force(s::SeriesSolve{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    arg.head == 0 || throw(ErrorException("non-zero head of argument in solve"))
    
    res = force(s.res)::Cons{T}
    return consmemo(res.head, SeriesSolve{T}(arg, res.tail / arg.tail))
end

force (generic function with 9 methods)

In [14]:
"Обращение ряда (в смысле подстановки)"
function inv(g::Series{T})::Series{T} where T <: Number
    return solve(g)
end

inv

In [42]:
"Поиск фиксированной точки, т.е. решение уравнения s = f(s)"
mutable struct SeriesFix{T <: Number} <: Series{T}
    fun::Union{Function, Series{T}}

    function SeriesFix{T}(fun::Function) where T <: Number
        return new(s -> s.fun = force(fun(s)))
    end
end


function force(s::SeriesFix{T})::Cons{T} where T <: Number
    return isa(s.fun, Function) ? s.fun(s) : s.fun
end

force (generic function with 13 methods)

In [52]:
struct SeriesDiff{T <: Number} <: Series{T}
    arg::Series{T}
    index::Int
end


"Дифференцирование"
function differentiate(s::Series{T})::Series{T} where T <: Number
    s = force(s).tail
    return SeriesDiff{T}(s, T(1))
end

    
function force(s::SeriesDiff{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    return consmemo(arg.head * T(s.index), SeriesDiff{T}(arg.tail, s.index + 1))
end

force (generic function with 13 methods)

In [51]:
struct SeriesInt{T <: Number} <: Series{T}
    arg::Series{T}
    index::Int
end


"Интегрирование"
function integrate(s::Series{T}, initial::T=T(0))::Series{T} where T <: Number
    return consmemo(initial, SeriesInt{T}(s, T(1)))
end


function force(s::SeriesInt{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    return consmemo(arg.head / T(s.index), SeriesInt{T}(arg.tail, s.index + 1))
end

force (generic function with 13 methods)

In [18]:
ser0 = generate(Rational{Int}, i -> i == 2 ? -1 : 0)

$0 + 0 \cdot x -1 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


In [19]:
ser1 = generate(Rational{Int}, i -> 1//1)

$1 + x + x^{2} + x^{3} + x^{4} + x^{5} + x^{6} + x^{7} + x^{8} + x^{9} + ...$


In [20]:
ser2 = generate(Rational{Int}, i -> (i == 0) ? 0 : (-1)^i // i)

$0 -1 \cdot x + \frac{1}{2} \cdot x^{2} + \frac{-1}{3} \cdot x^{3} + \frac{1}{4} \cdot x^{4} + \frac{-1}{5} \cdot x^{5} + \frac{1}{6} \cdot x^{6} + \frac{-1}{7} \cdot x^{7} + \frac{1}{8} \cdot x^{8} + \frac{-1}{9} \cdot x^{9} + ...$


In [21]:
ser1 + ser2

$1 + 0 \cdot x + \frac{3}{2} \cdot x^{2} + \frac{2}{3} \cdot x^{3} + \frac{5}{4} \cdot x^{4} + \frac{4}{5} \cdot x^{5} + \frac{7}{6} \cdot x^{6} + \frac{6}{7} \cdot x^{7} + \frac{9}{8} \cdot x^{8} + \frac{8}{9} \cdot x^{9} + ...$


In [22]:
1//3 * ser1

$\frac{1}{3} + \frac{1}{3} \cdot x + \frac{1}{3} \cdot x^{2} + \frac{1}{3} \cdot x^{3} + \frac{1}{3} \cdot x^{4} + \frac{1}{3} \cdot x^{5} + \frac{1}{3} \cdot x^{6} + \frac{1}{3} \cdot x^{7} + \frac{1}{3} \cdot x^{8} + \frac{1}{3} \cdot x^{9} + ...$


In [23]:
ser2 * ser2

$0 + 0 \cdot x + x^{2} -1 \cdot x^{3} + \frac{11}{12} \cdot x^{4} + \frac{-5}{6} \cdot x^{5} + \frac{137}{180} \cdot x^{6} + \frac{-7}{10} \cdot x^{7} + \frac{363}{560} \cdot x^{8} + \frac{-761}{1260} \cdot x^{9} + ...$


In [24]:
ser0(ser2)

$0 + 0 \cdot x -1 \cdot x^{2} + x^{3} + \frac{-11}{12} \cdot x^{4} + \frac{5}{6} \cdot x^{5} + \frac{-137}{180} \cdot x^{6} + \frac{7}{10} \cdot x^{7} + \frac{-363}{560} \cdot x^{8} + \frac{761}{1260} \cdot x^{9} + ...$


In [25]:
recip(ser1)

$1 + 0 \cdot x + 0 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


In [26]:
ser1 / ser1

$1 + x + x^{2} + x^{3} + x^{4} + x^{5} + x^{6} + x^{7} + x^{8} + x^{9} + ...$


In [27]:
ser1(ser0)

$1 + 0 \cdot x -1 \cdot x^{2} + 0 \cdot x^{3} + x^{4} + 0 \cdot x^{5} -1 \cdot x^{6} + 0 \cdot x^{7} + x^{8} + 0 \cdot x^{9} + ...$


In [28]:
ser2 / ser2

$1 + \frac{-1}{2} \cdot x + \frac{1}{3} \cdot x^{2} + \frac{-1}{4} \cdot x^{3} + \frac{1}{5} \cdot x^{4} + \frac{-1}{6} \cdot x^{5} + \frac{1}{7} \cdot x^{6} + \frac{-1}{8} \cdot x^{7} + \frac{1}{9} \cdot x^{8} + \frac{-1}{10} \cdot x^{9} + ...$


In [29]:
3//1 * ser1 * ser1

$3 + 6 \cdot x + 9 \cdot x^{2} + 12 \cdot x^{3} + 15 \cdot x^{4} + 18 \cdot x^{5} + 21 \cdot x^{6} + 24 \cdot x^{7} + 27 \cdot x^{8} + 30 \cdot x^{9} + ...$


In [30]:
ser1(ser2)

$1 -1 \cdot x + \frac{3}{2} \cdot x^{2} + \frac{-7}{3} \cdot x^{3} + \frac{11}{3} \cdot x^{4} + \frac{-347}{60} \cdot x^{5} + \frac{3289}{360} \cdot x^{6} + \frac{-1011}{70} \cdot x^{7} + \frac{38371}{1680} \cdot x^{8} + \frac{-136553}{3780} \cdot x^{9} + ...$


In [33]:
@time display(inv(inv(inv(inv(inv(inv(ser2)))))))

$0 -1 \cdot x + 0 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


  1.114013 seconds (3.29 M allocations: 151.994 MiB, 4.84% gc time, 88.03% compilation time)


In [53]:
FixSeries{Rational{Int}}(s -> integrate(s, 1//1))

$1 + x + \frac{1}{2} \cdot x^{2} + \frac{1}{6} \cdot x^{3} + \frac{1}{24} \cdot x^{4} + \frac{1}{120} \cdot x^{5} + \frac{1}{720} \cdot x^{6} + \frac{1}{5040} \cdot x^{7} + \frac{1}{40320} \cdot x^{8} + \frac{1}{362880} \cdot x^{9} + ...$


In [58]:
FixSeries{Rational{Int}}(s -> integrate(-integrate(s), 1//1))

$1 + 0 \cdot x + \frac{-1}{2} \cdot x^{2} + 0 \cdot x^{3} + \frac{1}{24} \cdot x^{4} + 0 \cdot x^{5} + \frac{-1}{720} \cdot x^{6} + 0 \cdot x^{7} + \frac{1}{40320} \cdot x^{8} + 0 \cdot x^{9} + ...$


In [59]:
FixSeries{Rational{Int}}(s -> integrate(integrate(-s, 1//1)))

$0 + x + 0 \cdot x^{2} + \frac{-1}{6} \cdot x^{3} + 0 \cdot x^{4} + \frac{1}{120} \cdot x^{5} + 0 \cdot x^{6} + \frac{-1}{5040} \cdot x^{7} + 0 \cdot x^{8} + \frac{1}{362880} \cdot x^{9} + ...$
